# ML2CPP

## Preparing the dataset

In [1]:
from sklearn import datasets
import numpy as np
import pandas as pd

def populate_table(tablename, feature_names):
    iris = datasets.load_iris()
    X = iris.data  
    N = X.shape[0]
    y = iris.target.reshape(N,1)
    k = np.arange(N).reshape(N, 1)
    k_X_y = np.concatenate((k, X, y) , axis=1)
    lTable=pd.DataFrame(k_X_y)
    # print(lTable.head())
    lTable.columns = ['idx'] + feature_names + ['TGT'];
    lTable['TGT'] = lTable['TGT'].apply(int)
    lTable['idx'] = lTable['idx'].apply(int)
    lTable.to_csv(tablename , float_format='%.14g')



In [2]:
metadata = {"primary_key" : "KEY",
            "features" : ['sepal_length_cm', 'sepal_width_cm', 'petal_length_cm', 'petal_width_cm'],
            "targets" : ["TGT"],
            "table" : "iris"}

In [3]:
populate_table("/tmp/iris.csv" , metadata["features"])


In [4]:
df = pd.read_csv("/tmp/iris.csv")
df.sample(12, random_state=1960)

,Unnamed: 0,idx,sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,TGT
114,114,114,5.8,2.8,5.1,2.4,2
74,74,74,6.4,2.9,4.3,1.3,1
9,9,9,4.9,3.1,1.5,0.1,0
88,88,88,5.6,3.0,4.1,1.3,1
25,25,25,5.0,3.0,1.6,0.2,0
5,5,5,5.4,3.9,1.7,0.4,0
48,48,48,5.3,3.7,1.5,0.2,0
117,117,117,7.7,3.8,6.7,2.2,2
83,83,83,6.0,2.7,5.1,1.6,1
105,105,105,7.6,3.0,6.6,2.1,2


## Training a Model

In [5]:


# train any scikit model on the iris dataset
import xgboost as xgb

clf = xgb.XGBClassifier(n_estimators=120, nthread=1, min_child_weight=10, max_depth=3, seed=1960);

clf.fit(df[metadata['features']].values, df[metadata['targets']].values)


/home/antoine/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=3,
              min_child_weight=10, missing=nan, monotone_constraints=None,
              n_estimators=120, n_jobs=1, nthread=1, num_parallel_tree=1,
              objective='multi:softprob', random_state=1960, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, seed=1960, subsample=1,
              tree_method=None, validate_parameters=False, verbosity=None)

## Deploying the Model

In [6]:

def generate_cpp_for_model(model):
    import pickle, json, requests, base64
    b64_data = base64.b64encode(pickle.dumps(model)).decode('utf-8')
    # send the model th the web service
    json_data={"Name":"model_cpp_sample", 
               "PickleData":b64_data , 
               "SQLDialect":"CPP",
               "FeatureNames" : metadata['features']}
    r = requests.post("https://sklearn2sql.herokuapp.com/model", json=json_data)
    content = r.json()
    lCPP = content["model"]["SQLGenrationResult"][0]["SQL"]
    # print(lCPP);
    return lCPP


lCPPCode = generate_cpp_for_model(clf);


In [7]:
print(lCPPCode)

namespace  {

	namespace XGB_Tree_0_0 {
	
		typedef std::vector<double> tNodeData;
		std::map<int, tNodeData> Decision_Tree_Node_data = {
				{ 1 ,  {0.430622011 }} ,
				{ 2 ,  {-0.220048919 }} 
		};
		
	
		int get_decision_tree_node_index(std::any Feature_0, std::any Feature_1, std::any Feature_2, std::any Feature_3) {
			int lNodeIndex = ( Feature_2 < 2.45000005 ) ? ( 1 ) : ( 2 );
		
			return lNodeIndex;
		}
		
	
		std::vector<std::string> get_input_names(){
			std::vector<std::string> lFeatures = { "Feature_0", "Feature_1", "Feature_2", "Feature_3" };
	
			return lFeatures;
		}
	
		std::vector<std::any> get_classes(){
			std::vector<std::any> lClasses = { 0, 1, 2 };
	
			return lClasses;
		}
	
		std::vector<std::string> get_output_names(){
			std::vector<std::string> lOutputs = { 
				"Score_0", "Score_1", "Score_2",
				"Proba_0", "Proba_1", "Proba_2",
				"LogProba_0", "LogProba_1", "LogProba_2" };
	
			return lOutputs;
		}
	
		tTable compute_classification_scores(std::any Featu

In [8]:
    def write_text_to_file(iCPPCode, oCPPFile):          
        with open(oCPPFile, "w") as text_file:
            text_file.write(iCPPCode)

    def add_cpp_main_function(iCPPCode, iCSVFile):
        lCPPCode = "#include \"Generic.i\"\n\n"
        lCPPCode = lCPPCode + iCPPCode
        lCPPCode = lCPPCode + "\tint main() {\n"
        lCPPCode = lCPPCode + "\t\tscore_csv_file(\"" + iCSVFile +"\");\n"
        lCPPCode = lCPPCode + "\treturn 0;\n}\n"
        return lCPPCode

    def compile_cpp_code_as_executable(iName):
        import subprocess
        lCommand = ["g++", "-Wall", "-Wno-unused-function", "-std=c++17" , "-g" ,  "-o", iName + ".exe",  iName + ".cpp"]
        print("EXECUTING" , "'" + " ".join(lCommand) + "'")
        result = subprocess.check_output(lCommand)
        # print(result)

    def execute_cpp_model(iName, iCSVFile):
        import subprocess
        result2 = subprocess.check_output([iName + ".exe",  iCSVFile])
        result2 = result2.decode()
        print(result2[:1000])
        print(result2[-1000:])
        return result2
        
    def execute_cpp_code(iCPPCode, iCSVFile):
        lName = "/tmp/sklearn2sql_cpp_" + str(id(clf));
        lCPPCode = add_cpp_main_function(iCPPCode, iCSVFile)
        write_text_to_file(lCPPCode, lName + ".cpp")
        compile_cpp_code_as_executable(lName)
        result = execute_cpp_model(lName, iCSVFile)
        write_text_to_file(str(result), lName + ".out")
        return lName + ".out"


In [9]:
populate_table("/tmp/iris2.csv" , ["Feature_0", "Feature_1", "Feature_2", "Feature_3"])
lCPPOutput = execute_cpp_code(lCPPCode , "/tmp/iris2.csv")
cpp_output = pd.read_csv(lCPPOutput)

EXECUTING 'g++ -Wall -Wno-unused-function -std=c++17 -g -o /tmp/sklearn2sql_cpp_140237989966560.exe /tmp/sklearn2sql_cpp_140237989966560.cpp'
idx,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision,DecisionProba
0,,,,0.91764759810273,0.04696863854605,0.03538376335122,-0.08594184212485,-3.05827516497005,-3.34150222645505,0,0.91764759810273
1,,,,0.91764759810273,0.04696863854605,0.03538376335122,-0.08594184212485,-3.05827516497005,-3.34150222645505,0,0.91764759810273
2,,,,0.91764759810273,0.04696863854605,0.03538376335122,-0.08594184212485,-3.05827516497005,-3.34150222645505,0,0.91764759810273
3,,,,0.91764759810273,0.04696863854605,0.03538376335122,-0.08594184212485,-3.05827516497005,-3.34150222645505,0,0.91764759810273
4,,,,0.91764759810273,0.04696863854605,0.03538376335122,-0.08594184212485,-3.05827516497005,-3.34150222645505,0,0.91764759810273
5,,,,0.91764759810273,0.04696863854605,0.03538376335122,-0.08594184212485,-3.05827516497005,-3.3415022264

In [10]:
cpp_output.sample(12, random_state=1960)

,idx,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision,DecisionProba
114,114,NaN,NaN,NaN,0.039291,0.028568,0.932142,-3.236766,-3.555477,-0.070271,2,0.932142
74,74,NaN,NaN,NaN,0.018106,0.960351,0.021543,-4.011507,-0.040456,-3.837713,1,0.960351
9,9,NaN,NaN,NaN,0.917648,0.046969,0.035384,-0.085942,-3.058275,-3.341502,0,0.917648
88,88,NaN,NaN,NaN,0.057462,0.874168,0.068369,-2.856623,-0.134483,-2.682829,1,0.874168
25,25,NaN,NaN,NaN,0.917648,0.046969,0.035384,-0.085942,-3.058275,-3.341502,0,0.917648
5,5,NaN,NaN,NaN,0.917648,0.046969,0.035384,-0.085942,-3.058275,-3.341502,0,0.917648
48,48,NaN,NaN,NaN,0.917648,0.046969,0.035384,-0.085942,-3.058275,-3.341502,0,0.917648
117,117,NaN,NaN,NaN,0.030969,0.032802,0.936228,-3.474759,-3.417251,-0.065896,2,0.936228
83,83,NaN,NaN,NaN,0.075838,0.414328,0.509834,-2.579152,-0.881098,-0.673670,2,0.509834
105,105,NaN,NaN,NaN,0.030969,0.032802,0.936228,-3.474759,-3.417251,-0.065896,2,0.936228


In [11]:
skl_outputs = pd.DataFrame()
X = df[metadata['features']].values
skl_output_key = pd.DataFrame(list(range(X.shape[0])), columns=['idx']);

skl_output_score = pd.DataFrame(columns=['Score_0', 'Score_1', 'Score_2']);
skl_output_proba = pd.DataFrame(clf.predict_proba(X), columns=['Proba_0', 'Proba_1', 'Proba_2'])
skl_output_log_proba = pd.DataFrame(columns=['LogProba_0', 'LogProba_1', 'LogProba_2'])
skl_output_decision = pd.DataFrame(clf.predict(X), columns=['Decision'])
skl_output = pd.concat([skl_output_key, skl_output_score, skl_output_proba, skl_output_log_proba, skl_output_decision] , axis=1)
skl_output.sample(12, random_state=1960)

,idx,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision
114,114,NaN,NaN,NaN,0.039291,0.028568,0.932141,NaN,NaN,NaN,2
74,74,NaN,NaN,NaN,0.018106,0.960351,0.021543,NaN,NaN,NaN,1
9,9,NaN,NaN,NaN,0.917648,0.046969,0.035384,NaN,NaN,NaN,0
88,88,NaN,NaN,NaN,0.057462,0.874168,0.068369,NaN,NaN,NaN,1
25,25,NaN,NaN,NaN,0.917648,0.046969,0.035384,NaN,NaN,NaN,0
5,5,NaN,NaN,NaN,0.917648,0.046969,0.035384,NaN,NaN,NaN,0
48,48,NaN,NaN,NaN,0.917648,0.046969,0.035384,NaN,NaN,NaN,0
117,117,NaN,NaN,NaN,0.030969,0.032802,0.936228,NaN,NaN,NaN,2
83,83,NaN,NaN,NaN,0.075838,0.414328,0.509834,NaN,NaN,NaN,2
105,105,NaN,NaN,NaN,0.030969,0.032802,0.936228,NaN,NaN,NaN,2


In [12]:
cpp_skl_join = skl_output.join(cpp_output , how='left', on='idx', lsuffix='_skl', rsuffix='_cpp')

In [13]:
cpp_skl_join.sample(12, random_state=1960)

,idx_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_1_cpp,Score_2_cpp,Proba_0_cpp,Proba_1_cpp,Proba_2_cpp,LogProba_0_cpp,LogProba_1_cpp,LogProba_2_cpp,Decision_cpp,DecisionProba
114,114,NaN,NaN,NaN,0.039291,0.028568,0.932141,NaN,NaN,NaN,...,NaN,NaN,0.039291,0.028568,0.932142,-3.236766,-3.555477,-0.070271,2,0.932142
74,74,NaN,NaN,NaN,0.018106,0.960351,0.021543,NaN,NaN,NaN,...,NaN,NaN,0.018106,0.960351,0.021543,-4.011507,-0.040456,-3.837713,1,0.960351
9,9,NaN,NaN,NaN,0.917648,0.046969,0.035384,NaN,NaN,NaN,...,NaN,NaN,0.917648,0.046969,0.035384,-0.085942,-3.058275,-3.341502,0,0.917648
88,88,NaN,NaN,NaN,0.057462,0.874168,0.068369,NaN,NaN,NaN,...,NaN,NaN,0.057462,0.874168,0.068369,-2.856623,-0.134483,-2.682829,1,0.874168
25,25,NaN,NaN,NaN,0.917648,0.046969,0.035384,NaN,NaN,NaN,...,NaN,NaN,0.917648,0.046969,0.035384,-0.085942,-3.058275,-3.341502,0,0.917648
5,5,NaN,NaN,NaN,0.917648,0.046969,0.035384,NaN,NaN,NaN,...,NaN,NaN,0.917648,0.046969,0.035384,-0.085942,-3.058275,-3.341502,0,0.917648
48,48,NaN,NaN,NaN,0.917648,0.046969,0.035384,NaN,NaN,NaN,...,NaN,NaN,0.917648,0.046969,0.035384,-0.085942,-3.058275,-3.341502,0,0.917648
117,117,NaN,NaN,NaN,0.030969,0.032802,0.936228,NaN,NaN,NaN,...,NaN,NaN,0.030969,0.032802,0.936228,-3.474759,-3.417251,-0.065896,2,0.936228
83,83,NaN,NaN,NaN,0.075838,0.414328,0.509834,NaN,NaN,NaN,...,NaN,NaN,0.075838,0.414328,0.509834,-2.579152,-0.881098,-0.673670,2,0.509834
105,105,NaN,NaN,NaN,0.030969,0.032802,0.936228,NaN,NaN,NaN,...,NaN,NaN,0.030969,0.032802,0.936228,-3.474759,-3.417251,-0.065896,2,0.936228


In [14]:
condition = (cpp_skl_join.Decision_cpp != cpp_skl_join.Decision_skl)
cpp_skl_join[condition]


,idx_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_1_cpp,Score_2_cpp,Proba_0_cpp,Proba_1_cpp,Proba_2_cpp,LogProba_0_cpp,LogProba_1_cpp,LogProba_2_cpp,Decision_cpp,DecisionProba
